# Investopedia Scraper

## Imports

This installations and imports are needed to run Selenium etc. in Google Colab.

In [1]:
def initial_install():
#  !pip install selenium
#  !apt-get update # to update ubuntu to correctly run apt install
#  !apt install firefox-geckodriver
#  !cp /usr/lib/geckodriver /usr/bin
#  !cp /usr/lib/firefox /usr/bin
  !pip install selenium
  !apt-get update # to update ubuntu to correctly run apt install
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  import sys
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

initial_install()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
import os
import json
import random
import csv
import re

# Selenium Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

# Import Firefox instance
from selenium.webdriver.firefox.options import Options

from IPython.display import clear_output

import pickle

import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Google Drive

In [3]:
#drive.mount('/content/drive')

## Definitions

In [4]:
def instantiate_driver():
    #options = Options()
    #options.add_argument('--headless')
    #options.add_argument('--ignore-certificate-errors')
    #options.add_argument('--incognito')
    #driver = webdriver.Firefox(options=options, executable_path='/usr/bin/geckodriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    return driver

def ByCheatSheet():
    by_values = [
            'ID = "id"',
            'XPATH = "xpath"',
            'LINK_TEXT = "link text"',
            'PARTIAL_LINK_TEXT = "partial link text"',
            'NAME = "name"',
            'TAG_NAME = "tag name"',
            'CLASS_NAME = "class name"',
            'CSS_SELECTOR = "css selector"',
    ]
    print(" \n".join(by_values))


## Scraping part
### Base Values

In [5]:
# Create dictonary of url basis
basedata = {
    'base' : 'https://www.investopedia.com/',
    'base_nr': 4769350,
    'direct': 'terms-beginning-with-{}-{}'}

# Create a list of letters in alphabet
alphabet = ['num','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
# Create iterator
keyword_url = [element for element in zip(alphabet, [num + basedata['base_nr'] for num in range(len(alphabet))])]
# create urls
urls = np.array([basedata['base'] + basedata['direct'].format(keyword.lower(), num) for keyword, num in keyword_url])

urls

array(['https://www.investopedia.com/terms-beginning-with-num-4769350',
       'https://www.investopedia.com/terms-beginning-with-a-4769351',
       'https://www.investopedia.com/terms-beginning-with-b-4769352',
       'https://www.investopedia.com/terms-beginning-with-c-4769353',
       'https://www.investopedia.com/terms-beginning-with-d-4769354',
       'https://www.investopedia.com/terms-beginning-with-e-4769355',
       'https://www.investopedia.com/terms-beginning-with-f-4769356',
       'https://www.investopedia.com/terms-beginning-with-g-4769357',
       'https://www.investopedia.com/terms-beginning-with-h-4769358',
       'https://www.investopedia.com/terms-beginning-with-i-4769359',
       'https://www.investopedia.com/terms-beginning-with-j-4769360',
       'https://www.investopedia.com/terms-beginning-with-k-4769361',
       'https://www.investopedia.com/terms-beginning-with-l-4769362',
       'https://www.investopedia.com/terms-beginning-with-m-4769363',
       'https://ww

In [6]:
print(os.getcwd())
base_path = '/content/drive/MyDrive/NaturalLanguageUnderstanding/homework/data'

with open(os.path.join(base_path,'hello.txt'),'w') as file:
  file.write('hello')

/content


### Scraper

In [ ]:
with open(os.path.join(base_path,'error_log.txt'),'w') as file:
  # Iterate through all possible urls for all letters (and number)
  for n,letter in enumerate(urls):
    # Instantiate driver instance
    driver_letter = instantiate_driver()
    # Call page (get request)
    driver_letter.get(letter)
    # Find all elements in overview
    class_elements = driver_letter.find_elements(By.XPATH, '//*[@id="dictionary-top300-list__list-content_1-0"]/*')
    # Collect all elements with tag name 'a'
    title_hrefs = np.array([(element.text,element.get_attribute('href')) for element in class_elements if element.tag_name == 'a'])

    # Creation of intermediate df 
    df_ = pd.DataFrame(
        {
            'title':np.array([element[0] for element in title_hrefs]),
            'href':np.array([element[1] for element in title_hrefs]), 
            'explanation':np.zeros(np.array([element[0] for element in title_hrefs]).shape),
            }
        )

    for m,term in enumerate(title_hrefs[:,1]):
      # Xpath of introduction text
      xpath = '//*[@id="mntl-sc-block_1-0-1"]'
      # Instantiate driver instance
      driver_term = instantiate_driver()
      # Call page (get request)
      driver_term.get(term)
      # Try and Except block for catching StaleElementReferenceException
      try:
        # Tries to find the respective text values. Waiting time is 10s
        explanation = WebDriverWait(driver_term, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).text
      except:
        # Dummy value for explanation - NA values for df
        explanation = 'No explanation found'
        # Print value for console and log.txt file
        error = f'********** href: {term} did not work **********'
        print(error)
        file.write(f"{letter.split('-')[3].upper()} | term: {term}\n") 

      # Assign value to intermediate df
      df_ = df_.assign(explanation = lambda x: np.where(x['href'] == term, explanation, x['explanation'])) 
    
  #    if df_[df_['href'] == term].explanation[0] == '0.0':
  #      print(f'********** href: {href} did not work **********') 

      # Closes driver for term (driver inner loop)
      driver_term.quit()

    # Console output for orientation
    print(f"Letter: {letter.split('-')[3]} - done")

    # Saves df as pickle file for post processing
    with open(os.path.join(base_path,f"{letter.split('-')[3].upper()}_investopedia_glossary_Pickle"), 'ab') as f:
      pickle.dump(df_, f)

    # Closes driver for letter (driver outer loop)
    driver_letter.quit()

Letter: num - done
********** href: https://www.investopedia.com/senior-care-4427723 did not work **********
********** href: https://www.investopedia.com/alternative-investments-4427781 did not work **********
Letter: a - done
Letter: b - done
********** href: https://www.investopedia.com/corporate-finance-4689819 did not work **********
Letter: c - done
********** href: https://www.investopedia.com/behavioral-economics-4689799 did not work **********
********** href: https://www.investopedia.com/business-essentials-4689832 did not work **********
********** href: https://www.investopedia.com/tech-stocks-4689742 did not work **********
Letter: d - done
Letter: e - done
Letter: f - done
********** href: https://www.investopedia.com/corporate-finance-and-accounting-4689821 did not work **********
********** href: https://www.investopedia.com/sectors-and-industries-analysis-4689756 did not work **********
********** href: https://www.investopedia.com/company-profiles-4689825 did not work